# Examining the Effects of DP on Exponential Smoothing Forecast Accuracy

***

In [1]:
# general modules
import pandas as pd
import numpy as np
import sktime

# import exponential smoothing forecasting model
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

# functions for transformation+forecasting pipeline
from sktime.forecasting.compose import TransformedTargetForecaster

# time series transformations
from sktime.transformations.series.detrend import ConditionalDeseasonalizer

##### the `helper_functions.py` file contains many custom functions we wrote to aid in our analysis
##### `full_coding_analysis` combines all of the following - train-test split data,
##### data protection, train models, compare accuracies, return accuracy results
from helper_functions import *

# suppress warnings from exponential smoothing model not converging
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import weekly finance time series
Y = np.genfromtxt("../../Data/Train/Clean/weekly_finance_clean.csv", delimiter = ',', skip_header = 1)
Y = pd.DataFrame(Y)

In [3]:
# detrender = Detrender()
# detrended_series = [detrender.fit_transform(series) for _ , series in Y.iterrows()]
# detrended_series = [i+np.abs(np.min(i))+1.0 for i in detrended_series]
# Y = pd.concat(detrended_series, axis=1).T

***

## SES

In [4]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        #("deseasonalize", ConditionalDeseasonalizer(model="additive", sp=52)),
        ("forecast", ExponentialSmoothing(use_boxcox=False)),
    ]
)

In [5]:
results_dict_ses = {}
fcasts_ses = {}
fcasts_protected_ses = {}
tests = {}
epsilons = [1, 10, 20]
horizons = [1, 20]

In [6]:
for e in epsilons:
    for h in horizons:
        idx = "h="+str(h)+", epsilon = "+str(e)
        results_dict_ses[idx], tests[idx], fcasts_ses[idx], fcasts_protected_ses[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             epsilon=e)

In [7]:
results_dict_ses

{'h=1, epsilon = 1': {'Mean Accuracies': array([2.18, 2.18]),
  'Protected Mean Accuracies:': array([126.75, 126.75]),
  '% Change Mean accuracy:': array([-5722.02, -5722.02]),
  '% Change Median accuracy:': array([-11907.38, -11907.38]),
  '% Forecasted Points adjusted downward:': 3.05,
  '% Forecasted Points adjusted upward:': 96.95,
  '% Series with improved accuracy:': array([0.61, 0.61]),
  '% Series with reduced accuracy:': array([99.39, 99.39]),
  'Original Mean Absolute Error Upward Adjusted:': 2.23,
  'Original Mean Absolute Error Downward Adjusted:': 0.35000000000000003,
  'Protected Mean Absolute Error Upward Adjusted:': 130.61,
  'Protected Mean Absolute Error Downward Adjusted:': 4.08},
 'h=20, epsilon = 1': {'Mean Accuracies': array([4.45, 5.43]),
  'Protected Mean Accuracies:': array([123.91, 124.  ]),
  '% Change Mean accuracy:': array([-2685.55, -2183.41]),
  '% Change Median accuracy:': array([-3620.84, -3013.39]),
  '% Forecasted Points adjusted downward:': 4.88,
  '

***
***

In [8]:
original_forecasts = fcasts_ses['h=20, epsilon = 20']
protected_forecasts = fcasts_protected_ses['h=20, epsilon = 20']
test = tests['h=20, epsilon = 20']

In [9]:
adjusted_up = original_forecasts < protected_forecasts
adjusted_up = pd.concat([row for i, row in adjusted_up.iterrows()])
adjusted_down = original_forecasts > protected_forecasts
adjusted_down = pd.concat([row for i, row in adjusted_down.iterrows()])

In [10]:
absolute_error_original = np.absolute(test - original_forecasts)
absolute_error_protected = np.absolute(test - protected_forecasts)

In [11]:
improved = absolute_error_original > absolute_error_protected
improved = pd.concat([row for i, row in improved.iterrows()])
worsened = absolute_error_original < absolute_error_protected
worsened = pd.concat([row for i, row in worsened.iterrows()])

In [12]:
np.mean(adjusted_down[improved])

0.4810126582278481

In [13]:
np.mean(adjusted_up[improved])

0.5189873417721519

***
***

## DES

In [14]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        #("deseasonalize", ConditionalDeseasonalizer(model="additive", sp=52)),
        ("forecast", ExponentialSmoothing(trend="additive", use_boxcox=False)),
    ]
)

In [15]:
results_dict_des = {}
fcasts_des = {}
fcasts_protected_des = {}
tests = {}
epsilons = [1, 10, 20]
horizons = [1, 20]

In [16]:
for e in epsilons:
    for h in horizons:
        idx = "h="+str(h)+", epsilon = "+str(e)
        results_dict_des[idx], tests[idx], fcasts_des[idx], fcasts_protected_des[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             epsilon=e)

In [17]:
results_dict_des

{'h=1, epsilon = 1': {'Mean Accuracies': array([2.1, 2.1]),
  'Protected Mean Accuracies:': array([127.45, 127.45]),
  '% Change Mean accuracy:': array([-5979.16, -5979.16]),
  '% Change Median accuracy:': array([-11624.59, -11624.59]),
  '% Forecasted Points adjusted downward:': 3.05,
  '% Forecasted Points adjusted upward:': 96.95,
  '% Series with improved accuracy:': array([0.61, 0.61]),
  '% Series with reduced accuracy:': array([99.39, 99.39]),
  'Original Mean Absolute Error Upward Adjusted:': 2.1399999999999997,
  'Original Mean Absolute Error Downward Adjusted:': 0.58,
  'Protected Mean Absolute Error Upward Adjusted:': 131.37,
  'Protected Mean Absolute Error Downward Adjusted:': 2.6100000000000003},
 'h=20, epsilon = 1': {'Mean Accuracies': array([4.28, 5.24]),
  'Protected Mean Accuracies:': array([124.08, 124.18]),
  '% Change Mean accuracy:': array([-2796.59, -2270.71]),
  '% Change Median accuracy:': array([-3746.7, -3015.3]),
  '% Forecasted Points adjusted downward:': 

***
***

## TES

In [18]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive",
                                          seasonal="additive",
                                          sp=52,
                                          damped_trend=False, 
                                          use_boxcox=False)),
    ]
)

In [19]:
results_dict_tes = {}
fcasts_tes = {}
fcasts_protected_tes = {}
tests = {}
epsilons = [1, 10, 20]
horizons = [1, 20]

In [20]:
for e in epsilons:
    for h in horizons:
        idx = "h="+str(h)+", epsilon = "+str(e)
        results_dict_tes[idx], tests[idx], fcasts_tes[idx], fcasts_protected_tes[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             epsilon=e)

In [21]:
results_dict_tes

{'h=1, epsilon = 1': {'Mean Accuracies': array([2.44, 2.44]),
  'Protected Mean Accuracies:': array([130.22, 130.22]),
  '% Change Mean accuracy:': array([-5232.81, -5232.81]),
  '% Change Median accuracy:': array([-11086.98, -11086.98]),
  '% Forecasted Points adjusted downward:': 3.66,
  '% Forecasted Points adjusted upward:': 96.34,
  '% Series with improved accuracy:': array([0., 0.]),
  '% Series with reduced accuracy:': array([100., 100.]),
  'Original Mean Absolute Error Upward Adjusted:': 2.5100000000000002,
  'Original Mean Absolute Error Downward Adjusted:': 0.72,
  'Protected Mean Absolute Error Upward Adjusted:': 134.67,
  'Protected Mean Absolute Error Downward Adjusted:': 13.19},
 'h=20, epsilon = 1': {'Mean Accuracies': array([4.75, 5.74]),
  'Protected Mean Accuracies:': array([122.57, 123.25]),
  '% Change Mean accuracy:': array([-2482.01, -2045.75]),
  '% Change Median accuracy:': array([-3423.77, -2758.52]),
  '% Forecasted Points adjusted downward:': 2.96,
  '% Fore